In [11]:
import db_builder.db_handler as dbh
import numpy as np
from torchvision import transforms, datasets
from tqdm import tqdm
import logging
from getpass import getpass
from PIL import Image
import torchvision.models as models
import torch.nn as nn
import torch
import os
import cv2
from collections import Counter

In [12]:
logging.getLogger('sqlalchemy.engine').propagate = False

db_params = {
    'user': 'postgres',
    'password': getpass('Please enter DB pw'),  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'ttdatabase',  #tensionTerminator
}
toolcheck = dbh.DB_Conn(db_params)
toolcheck.connect()
engine = toolcheck.get_engine()

Connected to PostgreSQL, DB: ttdatabase


In [13]:
unlabeled_loops = toolcheck.get_unlabeled_loop_id()

2023-11-28 19:29:56,205 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-28 19:29:56,205 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-28 19:29:56,205 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-28 19:29:56,206 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-28 19:29:56,207 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-28 19:29:56,208 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-28 19:29:56,209 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 19:29:56,209 INFO sqlalchemy.engine.Engine SELECT id FROM loop WHERE human_labeled = false
2023-11-28 19:29:56,210 INFO sqlalchemy.engine.Engine [generated in 0.00044s] {}


In [14]:
def time_formatter(total_seconds):

    hours = total_seconds // 3600
    remaining_seconds = total_seconds % 3600
    minutes = remaining_seconds // 60
    seconds = remaining_seconds % 60
    
    formatted_time = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    return formatted_time 

In [15]:
def trigger_crop(image):
    crop_box = (400, 450, 550, 550)
    cropped_image = transforms.functional.crop(image, *crop_box)
    return cropped_image

In [16]:
def video_to_image_converter(source_path: str, output_path: str, crop=False):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    #fps = int(vidcap.get(cv2.CAP_PROP_FPS))
    fps = 30

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                transform_test = transforms.ToPILImage()
                image = transform_test(image)
                if crop:
                    image = trigger_crop(image)
                image = np.asarray(image)
                cv2.imwrite(f"{output_path}/{count}.png", image)  # save frame as PNG file

        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [17]:
def checkImage(path: str, transfer_model, orig_set, transforms_wt):
    img = Image.open(path)
    img_tensor = transforms_wt(img).unsqueeze(0)
    prediction = transfer_model(img_tensor.to("cuda"))
    predicted_probabilities = torch.softmax(prediction, dim=1)
    predicted_class_idx = torch.argmax(prediction).item()
    predicted_class = orig_set.classes[predicted_class_idx]

    # Get the confidence score for the predicted class
    confidence = predicted_probabilities[0, predicted_class_idx].item() * 100  # Convert to percentage

    return predicted_class, confidence

In [18]:
def os_parent_dir():
    return os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

In [19]:
transforms_wt = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

tool_dataset = datasets.ImageFolder(
    root=os.path.join(os_parent_dir(),'data', 'tool_finder', '10_11_2023_21_05_33'),
    transform=transforms_wt
)

transfer_tool_model = models.resnet152()
transfer_tool_model.fc = nn.Sequential(
    nn.Linear(transfer_tool_model.fc.in_features, 2048),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1024, 500),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(500, 2)
)

transfer_tool_model_state_dict = torch.load(os.path.join(os_parent_dir(),'models', 'tool_finder', '10_11_2023_21_05_33' , 'model.pt'))
transfer_tool_model.load_state_dict(transfer_tool_model_state_dict)
transfer_tool_model.to("cuda")
transfer_tool_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [20]:
directory = "tmp_tools"
if not os.path.exists(directory):
    os.makedirs(directory)

for video_source_id in tqdm(unlabeled_loops):
    video_source = toolcheck.get_filepath_by_loop_id(video_source_id)
    video_to_image_converter(video_source, directory, True)
    dataset = []
    conf = []
    
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        predicted_class, confidence = checkImage(f, transfer_tool_model, tool_dataset, transforms_wt)
        conf.append(confidence)
        dataset.append(predicted_class)
    
        os.remove(f)
    os.rmdir(directory)
    dataset_confidence = sum(conf) / len(conf)
    tool_timer = Counter(dataset)
    for classes in tool_dataset.classes:
        toolcheck.set_tool_timer(video_source_id, classes,time_formatter(tool_timer[classes]))

    print(f"Loop_id: {video_source_id}, Result{Counter(dataset)}")

  0%|          | 0/22 [00:00<?, ?it/s]

2023-11-28 19:29:58,058 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-28 19:29:58,058 INFO sqlalchemy.engine.Engine [generated in 0.00052s] {'loop_id': 129}
2023-11-28 19:30:14,860 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop 
WHERE loop.id = %(id_1)s 
 LIMIT %(param_1)s
2023-11-28 19:30:14,861 INFO sqlalchemy.engine.Engine [generated in 0.00144s] {'id_1': 129, 'param_1': 1}
2023-11-28 19:30:14,864 INFO sqlalchemy.engine.Engine SELECT tools.id AS tools_id, tools.tool AS tools_tool, tools.time_in_use AS tools_time_in_use 
FROM tools, tools_loop_association 
WHERE %(param_1)s = tools_loop_association.loop_id AND tools.id = tools_loop_association.tool_id
2023-11-28 19:30:14,865 INFO sqlalchemy.engine.Engine [generated i

  5%|▍         | 1/22 [00:16<05:53, 16.85s/it]

Loop_id: 129, ResultCounter({'duoballs': 43, 'trigger': 35})
2023-11-28 19:30:14,905 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 19:30:14,906 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-28 19:30:14,907 INFO sqlalchemy.engine.Engine [cached since 16.85s ago] {'loop_id': 130}
2023-11-28 19:30:44,480 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop 
WHERE loop.id = %(id_1)s 
 LIMIT %(param_1)s
2023-11-28 19:30:44,481 INFO sqlalchemy.engine.Engine [cached since 29.62s ago] {'id_1': 130, 'param_1': 1}
2023-11-28 19:30:44,483 INFO sqlalchemy.engine.Engine SELECT tools.id AS tools_id, tools.tool AS tools_tool, tools.time_in_use AS tools_time_in_use 
FROM tools, tools_loop_association 
WHERE %(param_1)s = tools_l

  9%|▉         | 2/22 [00:46<08:07, 24.35s/it]

...

Loop_id: 148, ResultCounter({'duoballs': 47})
2023-11-28 19:39:37,262 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 19:39:37,262 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-28 19:39:37,263 INFO sqlalchemy.engine.Engine [cached since 579.2s ago] {'loop_id': 149}
2023-11-28 19:39:45,728 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop 
WHERE loop.id = %(id_1)s 
 LIMIT %(param_1)s
2023-11-28 19:39:45,729 INFO sqlalchemy.engine.Engine [cached since 570.9s ago] {'id_1': 149, 'param_1': 1}
2023-11-28 19:39:45,731 INFO sqlalchemy.engine.Engine SELECT tools.id AS tools_id, tools.tool AS tools_tool, tools.time_in_use AS tools_time_in_use 
FROM tools, tools_loop_association 
WHERE %(param_1)s = tools_loop_association

 95%|█████████▌| 21/22 [09:47<00:19, 19.08s/it]

Loop_id: 149, ResultCounter({'trigger': 35})
2023-11-28 19:39:45,764 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-28 19:39:45,765 INFO sqlalchemy.engine.Engine SELECT file_path FROM video WHERE loop_id = %(loop_id)s AND device = 'rgbCam'
2023-11-28 19:39:45,765 INFO sqlalchemy.engine.Engine [cached since 587.7s ago] {'loop_id': 150}
2023-11-28 19:40:14,374 INFO sqlalchemy.engine.Engine SELECT loop.id AS loop_id, loop.timestamp AS loop_timestamp, loop.session_length AS loop_session_length, loop.location AS loop_location, loop.human_labeled AS loop_human_labeled, loop.user_id AS loop_user_id 
FROM loop 
WHERE loop.id = %(id_1)s 
 LIMIT %(param_1)s
2023-11-28 19:40:14,375 INFO sqlalchemy.engine.Engine [cached since 599.5s ago] {'id_1': 150, 'param_1': 1}
2023-11-28 19:40:14,377 INFO sqlalchemy.engine.Engine SELECT tools.id AS tools_id, tools.tool AS tools_tool, tools.time_in_use AS tools_time_in_use 
FROM tools, tools_loop_association 
WHERE %(param_1)s = tools_loop_association.

100%|██████████| 22/22 [10:16<00:00, 28.02s/it]

Loop_id: 150, ResultCounter({'duoballs': 122})
